In [88]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

try:
    # Intenta establecer la conexión
    client = MongoClient('172.18.0.1', 27017)
    # Verifica si la conexión está disponible
    client.admin.command('ismaster')
    print("Conexión exitosa")

    # Seleccionar la base de datos
    db = client.supermercado
    # Seleccionar la colección
    collection = db.pedidos

    # Eliminar todos los documentos de la colección
    result = collection.delete_many({})

    print(f"Se han eliminado {result.deleted_count} documentos de la colección 'pedidos'.")

except ConnectionFailure:
    # Captura la excepción si la conexión falla
    print("Error al conectar con el servidor de MongoDB")


Conexión exitosa
Se han eliminado 0 documentos de la colección 'pedidos'.


In [126]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure
import json

try:
    # Intenta establecer la conexión
    client = MongoClient('172.18.0.1', 27017)
    # Verifica si la conexión está disponible
    client.admin.command('ismaster')
    print("Conexión exitosa")

    # Seleccionar la base de datos
    db = client.supermercado
    # Seleccionar la colección
    collection = db.pedidos

    # Abrir el archivo JSON y leer cada línea
    with open('ventas.json', 'r') as file:
        lines = file.readlines()
        for line in lines:
            # Cargar la línea como un objeto JSON
            data = json.loads(line)
            # Desanidar el campo _id y asignarlo directamente
            data['_id'] = data['_id']['$oid']
            # Insertar los datos en la colección
            collection.insert_one(data)

    print("Documentos importados correctamente.")

except ConnectionFailure:
    # Captura la excepción si la conexión falla
    print("Error al conectar con el servidor de MongoDB")


Conexión exitosa
Documentos importados correctamente.


In [127]:
num_documentos = collection.count_documents({})
print("Número total de registros importados:", num_documentos)


Número total de registros importados: 2999


In [109]:
pedido_47371 = collection.find_one({"ID_Pedido": 47371})
print(pedido_47371)


{'_id': '630a3d935f6bf41f76dcb2de', 'ID_Pedido': 47371, 'Importe total': 110.151, 'Fecha de compra': {'$date': '2020-10-14T00:00:00Z'}, 'Número de items': 2, 'Cliente': {'ID_Cliente': 54, 'Nombre': 'José', 'Apellido1': 'González', 'Apellido2': 'Álvarez', 'Dirección': {'ID_Dirección': 28, 'Calle': 'Avenida de Carlos', 'Número': 2, 'Piso': '1A', 'Código Postal': 45610, 'Localidad': 'Navalcán', 'Provincia': 'Toledo'}}}


In [110]:
pedido_47371_info = collection.find_one({"ID_Pedido": 47371}, {"_id": 0, "ID_Pedido": 1, "Importe total": 1})
print(pedido_47371_info)


{'ID_Pedido': 47371, 'Importe total': 110.151}


In [111]:
pedido_info = collection.find_one({"ID_Pedido": 47371}, {"_id": 1, "Cliente.Dirección.Código Postal": 1})
print(pedido_info)


{'_id': '630a3d935f6bf41f76dcb2de', 'Cliente': {'Dirección': {'Código Postal': 45610}}}


In [112]:
num_pedidos_sin_direccion = collection.count_documents({"Cliente.Dirección": {"$exists": False}})
print(num_pedidos_sin_direccion)

0


In [113]:
num_pedidos_con_cliente = collection.count_documents({"Cliente": {"$exists": True}})
print(num_pedidos_con_cliente)


2999


In [114]:
pedido_mayor_importe = collection.find_one(sort=[("Importe total", -1)])

if pedido_mayor_importe:
    id_pedido_mayor_importe = pedido_mayor_importe["ID_Pedido"]
    datos_cliente = pedido_mayor_importe["Cliente"]
    print("ID_Pedido:", id_pedido_mayor_importe)
    print("Datos del cliente:")
    print("ID_Cliente:", datos_cliente["ID_Cliente"])
    print("Nombre:", datos_cliente["Nombre"])
    print("Apellido1:", datos_cliente["Apellido1"])
    print("Apellido2:", datos_cliente["Apellido2"])
else:
    print("No se encontró ningún pedido.")


ID_Pedido: 79138
Datos del cliente:
ID_Cliente: 95
Nombre: Eduardo
Apellido1: Pérez
Apellido2: González


In [115]:
pedidos_con_mas_de_5_items = collection.distinct("ID_Pedido", {"Número de items": {"$gt": 5}})
print("ID_Pedido con más de 5 items:", pedidos_con_mas_de_5_items)

ID_Pedido con más de 5 items: [30521, 67569, 67693]


In [116]:
pedidos_cliente_1_40_45 = collection.find(
    {"Cliente.ID_Cliente": 1, "Importe total": {"$gte": 40, "$lte": 45}},
    {"ID_Pedido": 1, "Importe total": 1, "_id": 0}
)
for pedido in pedidos_cliente_1_40_45:
    print(pedido)

{'ID_Pedido': 60845, 'Importe total': 44.255}
{'ID_Pedido': 72047, 'Importe total': 41.612}
{'ID_Pedido': 45540, 'Importe total': 41.718}


In [117]:
num_pedidos_sin_numero = collection.count_documents({
    "Cliente.Dirección.Número": {"$exists": False}
})

num_pedidos_sin_piso = collection.count_documents({
    "Cliente.Dirección.Piso": {"$exists": False}
})

num_pedidos_sin_numero_o_piso = num_pedidos_sin_numero + num_pedidos_sin_piso

print("Número de pedidos con direcciones sin Número o Piso:", num_pedidos_sin_numero_o_piso)


Número de pedidos con direcciones sin Número o Piso: 2624


In [118]:
id_direcciones_distintas = collection.distinct("Cliente.Dirección.ID_Dirección", {
    "$or": [
        {"Cliente.Dirección.Número": {"$exists": False}},
        {"Cliente.Dirección.Piso": {"$exists": False}}
    ]
})

print("Valores distintos de ID_Dirección de los pedidos sin Número o Piso:", id_direcciones_distintas)


Valores distintos de ID_Dirección de los pedidos sin Número o Piso: [3, 6, 7, 8, 10, 11, 13, 16, 17, 19, 21, 25, 29, 31, 33, 35, 37, 45]


In [119]:
# 2. Identificar documentos que cumplen con las condiciones de actualización
# (Puedes realizar consultas para encontrar documentos que necesitan ser actualizados)
documents_to_update = collection.find({
    "Cliente.Dirección.Número": {"$exists": False}
})
print("\nDocumentos que necesitan actualización:")
for doc in documents_to_update:
    print(doc)

# 3. Diseñar consultas de actualización basadas en los datos revisados
# (Puedes crear consultas de actualización para corregir los datos faltantes)
update_query = {
    "$set": {"Cliente.Dirección.Incompleto": "SI"}
}

# Actualizar documentos que cumplen con la condición
result = collection.update_many(
    {"Cliente.Dirección.Número": {"$exists": False}},
    update_query
)
print(f"\nSe actualizaron {result.modified_count} documentos.")


Documentos que necesitan actualización:
{'_id': '630a3d935f6bf41f76dcb29c', 'ID_Pedido': 52865, 'Importe total': 61.583, 'Fecha de compra': {'$date': '2016-10-03T00:00:00Z'}, 'Número de items': 2, 'Cliente': {'ID_Cliente': 76, 'Nombre': 'Tomás', 'Apellido1': 'Gándara', 'Apellido2': 'Fernández', 'Dirección': {'Incompleto': 'SI'}}}
{'_id': '630a3d935f6bf41f76dcb2a1', 'ID_Pedido': 73667, 'Importe total': 14.878, 'Fecha de compra': {'$date': '2016-09-25T00:00:00Z'}, 'Número de items': 3, 'Cliente': {'ID_Cliente': 8, 'Nombre': 'Juan', 'Apellido1': 'Gándara', 'Apellido2': 'Álvarez', 'Dirección': {'Incompleto': 'SI'}}}
{'_id': '630a3d935f6bf41f76dcb2a9', 'ID_Pedido': 46269, 'Importe total': 106.09, 'Fecha de compra': {'$date': '2021-09-20T00:00:00Z'}, 'Número de items': 1, 'Cliente': {'Dirección': {'Incompleto': 'SI'}}}
{'_id': '630a3d935f6bf41f76dcb2ae', 'ID_Pedido': 77194, 'Importe total': 64.401, 'Fecha de compra': {'$date': '2021-10-11T00:00:00Z'}, 'Número de items': 2, 'Cliente': {'ID_C

In [120]:
from pymongo import MongoClient


id_direcciones = [3, 6, 7, 8, 10, 11, 13, 16, 17, 19, 21, 25, 29, 31, 33, 35, 37, 45]

result = collection.update_many(
    {
        "Cliente.Dirección.ID_Dirección": {"$in": id_direcciones},
        "$or": [
            {"Cliente.Dirección.Número": {"$exists": False}},
            {"Cliente.Dirección.Piso": {"$exists": False}}
        ]
    },
    {
        "$set": {"Cliente.Dirección.Incompleto": "SI"}
    }
)

# Imprimir el número de documentos actualizados
print("Se actualizaron", result.modified_count, "documentos.")


Se actualizaron 0 documentos.


In [121]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

try:
    client = MongoClient('172.18.0.1', 27017)
    client.admin.command('ismaster')
    print("Conexión exitosa")

    db = client.supermercado
    # Seleccionar la colección
    collection = db.pedidos

    result = collection.update_many(
        {"Dirección.Calle": "Avenida de Casablanca 45"},
        {"$set": {"Dirección.Calle": "Avenida de Casablanca"}}
    )

    print(f"Se han actualizado {result.modified_count} documentos.")

except ConnectionFailure:
    print("Error al conectar con el servidor de MongoDB")


Conexión exitosa
Se han actualizado 0 documentos.


In [122]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

try:
    client = MongoClient('172.18.0.1', 27017)
    client.admin.command('ismaster')
    print("Conexión exitosa")

    db = client.supermercado
    collection = db.pedidos

    mismatched_addresses = collection.find({
        "Cliente.Dirección": {"$exists": True},
        "$expr": {"$ne": ["$Cliente.Dirección.Localidad", "$Cliente.Dirección.Provincia"]}
    }, {"Cliente.Dirección.ID_Dirección": 1})

    mismatched_address_ids = [doc["Cliente"]["Dirección"]["ID_Dirección"] for doc in mismatched_addresses]

    print("ID de dirección donde la localidad y la provincia no coinciden:")
    for address_id in mismatched_address_ids:
        print(address_id)

except ConnectionFailure:
    print("Error al conectar con el servidor de MongoDB")


Conexión exitosa
ID de dirección donde la localidad y la provincia no coinciden:
10
10
29
8
10
14
42
42
11
29
2
42
21
14
42
35
2
29
14
10
28
10
40
8
10
39
39
40
35
28
14
34
39
35
40
21
42
8
35
35
39
10
39
35
45
8
21
42
12
8
10
35
24
35
14
2
35
12
28
2
2
42
11
40
28
34
8
8
2
34
14
2
42
29
40
24
29
2
10
45
40
35
42
2
2
10
35
3
40
29
8
34
35
2
8
35
42
35
39
8
35
34
35
8
45
8
8
2
10
35
2
39
12
40
35
28
2
11
2
35
40
8
2
35
34
12
35
45
2
28
39
42
3
2
34
8
10
39
24
39
35
8
40
45
10
21
8
40
8
2
39
10
42
2
34
11
39
39
34
2
45
42
42
39
24
28
10
2
29
34
2
42
2
34
3
39
8
35
10
34
39
11
39
42
12
42
8
2
34
40
12
29
2
2
42
39
2
3
35
2
35
40
10
2
39
35
24
35
8
14
14
39
35
12
14
10
34
8
24
12
42
40
35
12
42
8
21
12
12
34
40
10
40
28
28
28
12
21
10
42
28
10
14
11
2
2
40
39
40
10
14
8
42
8
14
42
10
24
10
35
29
12
2
21
12
10
39
42
12
12
39
2
29
10
10
10
3
21
28
42
28
10
45
12
45
28
3
21
35
12
35
3
39
10
39
10
39
12
29
2
3
29
42
45
29
10
40
39
35
11
40
14
3
2
12
35
11
39
42
24
29
2
12
24
2
42
14
35
40
12
1

In [123]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

try:
    client = MongoClient('172.18.0.1', 27017)
    client.admin.command('ismaster')
    print("Conexión exitosa")

    db = client.supermercado
    collection = db.pedidos

    orders_with_tomatoes = collection.find({"Items.Nombre": "Tomates"})

    print("Pedidos que contienen algún ítem con nombre 'Tomates':")
    for order in orders_with_tomatoes:
        print(order)

except ConnectionFailure:
    print("Error al conectar con el servidor de MongoDB")


Conexión exitosa
Pedidos que contienen algún ítem con nombre 'Tomates':
{'_id': '630a3d935f6bf41f76dcb2d3', 'ID_Pedido': 46312, 'Importe total': 106.305, 'Fecha de compra': {'$date': '2017-09-04T00:00:00Z'}, 'Número de items': 3, 'Cliente': {'ID_Cliente': 27, 'Nombre': 'Eduardo', 'Apellido1': 'Aspas', 'Apellido2': 'Míguez', 'Dirección': {'Incompleto': 'SI'}}, 'Items': [{'Nombre': 'Tomates', 'Importe unitario': 2.12, 'Cantidad': 1}, {'Nombre': 'Leche', 'Importe unitario': 1.12, 'Cantidad': 12}, {'Nombre': 'Mermelada', 'Importe unitario': 2.32, 'Cantidad': 2}]}
{'_id': '630a3d935f6bf41f76dcb39b', 'ID_Pedido': 14292, 'Importe total': 103.769, 'Fecha de compra': {'$date': '2021-08-23T00:00:00Z'}, 'Número de items': 1, 'Items': [{'Nombre': 'Tomates', 'Importe unitario': 4.98, 'Cantidad': 2}], 'Cliente': {'Dirección': {'Incompleto': 'SI'}}}


In [124]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

try:
    client = MongoClient('172.18.0.1', 27017)
    client.admin.command('ismaster')
    print("Conexión exitosa")

    db = client.supermercado
    collection = db.pedidos

    order_ids = collection.distinct("ID_Pedido", {"Items.Nombre": {"$all": ["Pan", "Leche"]}})

    print("IDs de Pedido que contienen 'Pan' y 'Leche' entre sus ítems:")
    for order_id in order_ids:
        print(order_id)

except ConnectionFailure:
    print("Error al conectar con el servidor de MongoDB")


Conexión exitosa
IDs de Pedido que contienen 'Pan' y 'Leche' entre sus ítems:
10074
30486
70812
79159
87049


In [125]:
from pymongo import MongoClient

# Establecer conexión con la base de datos
client = MongoClient('172.18.0.1', 27017)
database = client['nombre_de_tu_base_de_datos']  # Reemplaza 'nombre_de_tu_base_de_datos' con el nombre real de tu base de datos

# Contar documentos en la colección "pedidos"
coleccion_pedidos = database['pedidos']
numero_de_documentos = coleccion_pedidos.count_documents({})

print("Número de documentos en la colección 'pedidos':", numero_de_documentos)


Número de documentos en la colección 'pedidos': 0
